In [144]:
import numpy as np
from dipy.io.streamline import load_trk
from dipy.viz import window, actor

In [145]:
def show_streamlines(streamlines): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamlines))
    window.show(ren)

In [146]:
def show_multiple_streamlines(streamline1,streamline2): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamline1))
    ren.add(actor.line(streamline2,colors=(0,1,1)))
    window.show(ren)

 load brain (tractogram)

In [147]:
brain, b_hr = load_trk("brain.trk")

load Arcuate Fasciculus left fiber bundle (AF_L)

In [148]:
bundle, bn_hr = load_trk("AF_L.trk")

In [149]:
#cst, bn_hr = load_trk("CST_recognized.trk")

let's visualize them

In [150]:
# first brain

#show_streamlines(brain)

In [151]:
# now bundle

#show_streamlines(bundle)

In [152]:
#show_streamlines(cst)

In [153]:
#show_multiple_streamlines(bundle,cst)

let's see its data type

In [154]:
#bundle # whole bundle, made of several streamlines

In [155]:
((brain)[0]).shape# number of streamlines in bundle

(331, 3)

In [156]:
# bundle[1] #one streamline in bundle
(np.array(bundle)[0]).shape
# np.array(bundle)

(331, 3)

In [157]:
(bundle[1].shape) #one point on one streamline

(557, 3)

In [158]:
npbrain = np.array(brain)

In [159]:
npbrain[0].shape

(331, 3)

In [160]:
len(bundle)

2000

these are x, y and z co-ordinates of the point

In [161]:
# set labels
labels = np.zeros(len(brain))
for i in range(len(bundle)):
    for j in range(len(brain)):
        if((npbrain[j].shape[0]==bundle[i].shape[0])):
#         print(i,j)
            if np.allclose(npbrain[j], bundle[i]):
                labels[j] = 1
                break
    if(i%100==0):
        print(i,end=" ")
                

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 

In [162]:
brain[0][1]

array([-53.0558  ,  16.419182,  44.916023], dtype=float32)

In [163]:
bundle[0][1]

array([-53.0558  ,  16.419182,  44.916023], dtype=float32)

In [164]:
# force put bundle data into the brain data
# for i in range(len(bundle)):
#     npbrain[i] = bundle[i]

In [165]:
# set labels
# labels = np.zeros(len(brain))
# for i in range(len(bundle)):
#     for j in range(len(brain)):
#         if((npbrain[j].shape[0]==bundle[i].shape[0])):
# #         print(i,j)
#             if np.allclose(npbrain[j], bundle[i]):
#                 print(j)
#                 labels[j] = 1
#                 break
#     if(i%1==0):
#         print("Not: "+ str(i))

In [166]:
npbrain.shape

(144000,)

In [167]:
n_pts = [len(streamline) for streamline in bundle]

In [168]:
from dipy.tracking.streamline import (set_number_of_points, nbytes,select_random_set_of_streamlines)
newStreamlines = set_number_of_points(bundle,101)
n_pts_ds = [len(streamline) for streamline in newStreamlines]

In [169]:
newBrain = set_number_of_points(brain,101)

In [170]:
newBrain[0].shape

(101, 3)

In [173]:
len(npbrain)

(144000,)